In [1]:
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn')
import json

In [2]:
init_types = ['dueweight', 'due', 'dueinitprodtime']
maint_types = ['greedy', 'rij', 'distributed', 'random', 'all']  # 'notabu',

types = []  # 'init_prod_time_notabu'
type_pairs = [[], []]
for init_type in init_types:
    for maint_type in maint_types:
        types.append(f'{init_type}_{maint_type}')
        type_pairs[0].append(init_type)
        type_pairs[1].append(maint_type)
types.append('gurobi')
type_pairs[0].append('gurobi')
type_pairs[1].append('gurobi')

In [14]:
def read_data(level):
    file_name = 'tabu/0811/' #'0701_2020/'  # 0512_allscen
    record = f'_upstream_{level}_obj'
    for type in types[:-1]:
        globals()[type + f'_upstream_{level}_df'] = np.maximum(pd.DataFrame(
            pd.read_csv(file_name + type + record + '.csv')), 1)

    gurobi_file_name = f'model/0811upstream_{level}_30min'   #'model/0627_30min, 0702U_30min'
    gurobi_df = np.maximum(pd.DataFrame(pd.read_csv(gurobi_file_name + '_bestbd' + '.csv')).iloc[:30, :], 1)
    return gurobi_df

In [53]:
def gap(level):
    gurobi_df = read_data(level=level)

    ratio_avg = []
    ratio_sig = []
    for type in types[:-1]:
        ratio_avg.append(np.average(np.maximum(globals()[type + f'_upstream_{level}_df'].values-gurobi_df.values, 0)/(globals()[type + f'_upstream_{level}_df'].values), axis=0))
        ratio_sig.append(np.var(np.maximum(globals()[type + f'_upstream_{level}_df'].values-gurobi_df.values, 0)/(globals()[type + f'_upstream_{level}_df'].values), axis=0))

    ratio_avg_df = pd.DataFrame(ratio_avg, columns=[f'avg_{level}'])
    ratio_avg_df.index = Series(types[:-1])
    ratio_sig_df = pd.DataFrame(ratio_sig, columns=[f'sig_{level}'])
    ratio_sig_df.index = Series(types[:-1])
    return ratio_avg_df, ratio_sig_df

In [54]:
avg_H, sig_H = gap(level='H')
avg_L, sig_L = gap(level='L')

gap_df = pd.concat([avg_H, sig_H, avg_L, sig_L], axis=1)
gap_df

,avg_H,sig_H,avg_L,sig_L
dueweight_greedy,0.082002,0.025871,0.128463,0.034908
dueweight_rij,0.159566,0.048021,0.248098,0.055854
dueweight_distributed,0.187804,0.053450,0.271679,0.063062
dueweight_random,0.140294,0.045124,0.232373,0.056804
dueweight_all,0.254544,0.077586,0.343858,0.087581
due_greedy,0.082002,0.025871,0.125272,0.034386
due_rij,0.159736,0.047892,0.248403,0.055938
due_distributed,0.180436,0.051158,0.268899,0.062494
due_random,0.159043,0.053520,0.237789,0.059729
due_all,0.254544,0.077586,0.343858,0.087581


In [40]:
### TIME
level='L'
file_name = f'model/0811upstream_{level}_30min' #'tabu/0701_2020/'#'model/0627_30min, 0702U_30min' #'tabu/0512_allscen/'
record = '_runtime'
type = '' #'dueweight_greedy'

runtime_df = pd.DataFrame(pd.read_csv(file_name + type + record + '.csv'))
runtime_df.mean()

upstreamdiscount_L    1040.177947
dtype: float64

In [84]:
level='L'
file_name = 'tabu/0811/' #'tabu/0701_2020/'#'model/0627_30min' #'tabu/0512_allscen/'
record = f'_upstream_{level}_runtime'
type = 'our'

taburuntime_df = pd.DataFrame(pd.read_csv(file_name + type + record + '.csv'))
taburuntime_df.mean()

upstreamdiscount_L    136.475088
dtype: float64

In [87]:
pd.DataFrame([['Base',0.0919,0.0372,164.4726,958.6667,],
    ['BL  ',0.2335,0.0910,133.8922,1229.4807,],
    ['BS  ',0.1025,0.0298,170.0120,1254.1857,],
    ['DDL ',0.0898,0.0295,149.9044,1001.8296,],
    ['DDS ',0.1081,0.0446,156.9013,1287.5891,],
    ['TPDL',0.1590,0.0506,142.7655,1131.9143,],
    ['TPDS',0.1739,0.0540,142.9252,1253.5995,],
    ['OU  ',0.7502,0.0044,175.8998,1800.0891,],
    ['ONU ',0.0121,0.0042,112.2693,311.3606,],
    ['MLDL',0.1402,0.0535,132.9439,1095.4576,],
    ['MLDS',0.1167,0.0505,165.1104,1178.0622,],
    ['DPSL',0.0820,0.0259,139.4014,1174.9128,],
    ['DPSS',0.1285,0.0349,136.4751,1040.1779,]], columns=['scen', 'avg', 'sig', 'time', 'MIP_time']).mean().round(4)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  del sys.path[0]


avg            0.1683
sig            0.0392
time         147.9210
MIP_time    1132.1020
dtype: float64

### Tabu

In [55]:
def tabu_gap(level):
    gurobi_df = read_data(level=level)

    ratio_avg = []
    ratio_sig = []
    for type in types[:-1]:
        ratio_avg.append(np.average((globals()[type + f'_upstream_{level}_df'].values)/(globals()[f'dueweight_greedy_upstream_{level}_df'].values), axis=0))
        ratio_sig.append(np.var((globals()[type + f'_upstream_{level}_df'].values)/(globals()[f'dueweight_greedy_upstream_{level}_df'].values), axis=0))

    ratio_avg_df = pd.DataFrame(ratio_avg, columns=[f'avg_{level}'])
    ratio_avg_df.index = Series(types[:-1])
    ratio_sig_df = pd.DataFrame(ratio_sig, columns=[f'sig_{level}'])
    ratio_sig_df.index = Series(types[:-1])
    
    return ratio_avg_df, ratio_sig_df

In [74]:
level='H'

tabu_avg_H, tabu_sig_H = tabu_gap(level='H')
tabu_avg_L, tabu_sig_L = tabu_gap(level='L')

tabu_gap_df = pd.concat([tabu_avg_H, tabu_sig_H, tabu_avg_L, tabu_sig_L], axis=1)
tabu_gap_df.loc[:, ['avg_H', 'avg_L']].round(2)

,avg_H,avg_L
dueweight_greedy,1.00,1.00
dueweight_rij,6.13,6.13
dueweight_distributed,6.53,7.45
dueweight_random,4.24,4.04
dueweight_all,15.62,15.62
due_greedy,1.04,3.48
due_rij,6.13,6.13
due_distributed,9.54,7.06
due_random,2.25,3.99
due_all,15.62,15.62


In [76]:
tabu_gap_df['init_type'] = type_pairs[0][:-1]
tabu_gap_df['maint_type'] = type_pairs[1][:-1]
dict_init = {"dueweight": "1_WEDD", "due": "2_EDD", "dueinitprodtime": "3_LSTR"}
dict_maint = {"greedy": "1_greedy", "rij": "2_profit-oriented", "distributed": "3_distributed",
              "random": "4_random", "all": "5_earliest"}
tabu_gap_df_2 = tabu_gap_df.replace({"init_type": dict_init, "maint_type": dict_maint}).round(2)

In [79]:
tabu_gap_df_2.iloc[:, :].groupby('init_type').mean().round(2)  #.sort_values(['base_case'])

,avg_H,sig_H,avg_L,sig_L
init_type,,,,
1_WEDD,6.70,693.75,6.85,696.80
2_EDD,6.92,722.72,7.26,722.48
3_LSTR,6.55,685.99,7.49,787.07


In [82]:
tabu_gap_df_2.iloc[:, :].groupby('maint_type').mean().round(2)  #.sort_values(['base_case'])

,avg_H,sig_H,avg_L,sig_L
maint_type,,,,
1_greedy,1.03,0.02,1.84,58.75
2_profit-oriented,6.13,406.01,6.13,406.00
3_distributed,7.43,482.92,7.80,476.47
4_random,3.41,80.20,4.60,201.14
5_earliest,15.62,2534.95,15.62,2534.89


In [ ]:

file_name = 'tabu/0811/' #'0701_2020/'  # 0512_allscen
data_name = 'dueweight_greedy_noswapping_obj'
noswapping_df = np.maximum(pd.DataFrame(pd.read_csv(file_name + data_name + '.csv')), 1)


ratio_avg.append(np.average(np.maximum(globals()[type + f'_upstream_{level}_df'].values-gurobi_df.values, 0)/(globals()[type + f'_upstream_{level}_df'].values), axis=0))
ratio_sig.append(np.var(np.maximum(globals()[type + f'_upstream_{level}_df'].values-gurobi_df.values, 0)/(globals()[type + f'_upstream_{level}_df'].values), axis=0))

ratio_avg_df = pd.DataFrame(ratio_avg, columns=[f'avg_{level}'])
ratio_avg_df.index = Series(types[:-1])
ratio_sig_df = pd.DataFrame(ratio_sig, columns=[f'sig_{level}'])
ratio_sig_df.index = Series(types[:-1])
 